In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [13]:
df_done = pd.read_table('data.tsv',header=None,index_col=0)
df_done = df_done.dropna(how='any')
df_done

,1,2,3,4
0,,,,
0,女性が逃げてしまう、事実とは反する技術職に関する噂を紹介している,キャリアの道は1つに絞られる、数学が得意でないといけない,仕事と生活を両立できない、1人で働く、オタクと働かなければならないの5つ,それはなぜでしょう？ 技術者は、働く企業や環境によっては、かなりの高収入であるケースも多いで...
1,より良い人間関係をつくるためのシンプルな方法を紹介している,相手に期待しすぎず、受け取るよりも与える方が多いくらいの気持ちでいる,親切な行動は他者に強い影響を与えるため、寛大な心で接すると良いとのこと,人間関係は基本的にギブ・アンド・テイクの法則に基づいています。他人と関わるのは、相手が何らか...
2,就寝前にスマホを見ないようにする方法を紹介している,日中にスマホの通知を意識的に無視する訓練をするのが効果的という,決めた時間にだけ確認することで、脳の神経伝達物質を訓練し直す,もし就寝前になるべく携帯を見ないようにしたいなら、日中に携帯の通知を意識的に無視する訓練をす...
3,犬にとって有害な食べ物をリストで紹介している,クルミやマカデミアナッツは、神経系と筋肉を損傷させてしまう,ブドウやレーズンは急性腎不全の原因になるなども,基本的に人間が食べるものはどれもペットにとっては有害になる可能性がある、と覚えておきましょう...
5,長時間労働で起きる悪影響が、科学的に実証された,ハーバード大学らの研究によると、死亡率を20％押し上げるという,その他、体調不良になるケースが約90％増加するなどの研究結果もある,他にも似たような研究結果があります。・仕事と家庭のバランスが取れずに体調不良になるケース（自...
...,...,...,...,...
54510,良品計画は13日、無印良品ネットストアで不正ログインがあったと発表した,不正ログインを受けた期間は7日から12日で、ID数は2万957件となった,商品購入やポイントの不正利用などは、現時点では確認していないという,同社が不正ログイン試行を受けた期間は、7日16時34分〜12日10時52分で、試行回数は42...
54513,スマートフォンの満足度が調査された,総合評価はソフトバンクが1位となり、2位以降はKDDI、NTTドコモと続いた,ソフトバンクとドコモの点数差は縮まったが、要因にiPhoneの販売が挙がった,携帯電話とスマートフォンの所有状況を尋ねたところ、「従来型の携帯電話(フィーチャーフォン)を...
54514,20〜30代の273人に、ITエンジニアの働き方に関するアンケート調査を行った,絶対に避けたい職場環境では、「効率が悪く仕事に終わりが見えない」が最多,「上司のITに不見識」「効率でなく労働時間で評価される」などが続いた,同調査の対象者となったのは「SIer・受託開発」などソフトウェア開発会社に勤めるエンジニアが...


In [16]:
done_list = df_done.index.values

In [17]:
len(done_list)

17908

In [2]:
df = pd.read_csv('data/train1.csv',header=None)
id_list = df[3]


In [ ]:
with open('data3.tsv','w') as f:
    for idx, page_id in enumerate(id_list):
        try:
            time.sleep(1)
            if '.' in page_id:
                page_id = page_id.split('.')[0]
            print(idx,'/',len(id_list), page_id)
            url = "https://news.livedoor.com/article/detail/{}/".format(page_id)
            response = requests.get(url)
            response.encoding = response.apparent_encoding
            bs = BeautifulSoup(response.text, 'html.parser')
            
            if bs.find('p', class_='topicsTxt') is not None:
                if bs.find('p', class_='topicsTxt').text == '提供社の都合により、削除されました。概要のみ掲載しております。':
                    print('提供社の都合により、削除されました。概要のみ掲載しております。')
                    continue

            data=[str(idx)]
            #summary
            print('summary')
            bs_ul = bs.find('ul', class_='summaryList')
            for bs_li in bs_ul.find_all('li'):
                data.append(bs_li.text)
            if len(data)!=4:
                continue
            #body
            print('body')
            body=[]
            bs_body = bs.find('div', class_='articleBody').find('span')
            if bs_body.find_all('p') == []:
                body.append(bs_body.text)
            else:
                for bs_p in bs_body.find_all('p'):
                    if bs_p.find('a') is None:
                        body.append(bs_p.text)
            script_str = 'varamp_med=\'2000689\';varamp_site=\'2000850\';varamp_frame=\'2007958\';varamp_rurl=document.referrer;varamp_send=location.protocol+\'//ads.adjust-net.jp/adserver/ad/ads_v.js?\'+Math.random();document.write(\"<scr\"+\"ipttype=\'text/javascript\'src=\'\"+amp_send+\"\'></scr\"+\"ipt>\");'
            data.append(''.join(body).replace(script_str,''))
            data = [txt.replace('\r','').replace('\t','').replace('\n','').replace(' ','') for txt in data]
#             f.write('\t'.join(data)+'\n')
            print('\t'.join(data))
        except Exception as e:
            print(url)
            print(e)
            continue
